## Create LMDB Dataset

In [ ]:
try:
    import lmdb
except ImportError:
    %pip install lmdb
    import lmdb

try:
    from tqdm import tqdm
except ImportError:
    %pip install tqdm
    from tqdm import tqdm
try:
    import fire
except ImportError:
    %pip install fire
    import fire

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=ace641c5264ae18e90324e8b24a4b7e6febc209b2c2af20d924febee1d04ef99
  Stored in directory: /root/.cache/pip/wheels/5b/eb/43/7295e71293b218ddfd627f935229bf54af9018add7fbb5aac6
Successfully built fire


In [ ]:
# save dir 생성
train_dir = 'train_lmdb'
val_dir = 'val_lmdb'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
def write_cache(env, cache):
    with env.begin(write=True) as txn:
        for k, v in cache.items():
            txn.put(k, v)

In [ ]:
def write_lmdb(df, env, bar_desc='train'):
    cnt = 1
    cache = {}
    for i in tqdm(range(len(df)), desc=bar_desc):
        img_path = df.iloc[i]['img_path']
        label = df.iloc[i]['label']
        
        with open(img_path, 'rb') as f:
            img_bin = f.read()
        
        image_key = f'image-{cnt:09d}'.encode()
        label_key = f'label-{cnt:09d}'.encode()

        cache[image_key] = img_bin
        cache[label_key] = label.encode()

        if cnt % 1000 == 0:
            write_cache(env, cache)
            cache = {}
        cnt += 1

    # remain cache
    cache['num-samples'.encode()] = str(cnt-1).encode()
    write_cache(env, cache)

In [ ]:
def create_and_write_lmdb():
    env_train = lmdb.open(train_dir, map_size=1099511627776)
    env_val = lmdb.open(val_dir, map_size=1099511627776)

    write_lmdb(train, env_train, bar_desc='train')
    write_lmdb(val, env_val, bar_desc='val'))

    env_train.close()
    env_val.close()

In [ ]:
fire.Fire(create_and_write_lmdb, command='')

test: 100%|██████████| 74121/74121 [00:19<00:00, 3864.93it/s]


## 생성이 잘 되었는지 체크

In [ ]:
import cv2
import matplotlib.pyplot as plt

lmdb.open(test_dir, readonly=True, lock=False).stat()

# lmdb read
env = lmdb.open(test_dir, readonly=True, lock=False)
txn = env.begin()
label = txn.get('label-000000001'.encode()).decode()
img = cv2.imdecode(np.frombuffer(txn.get('image-000000001'.encode()), dtype=np.uint8), cv2.IMREAD_COLOR)
print(label)
plt.imshow(img)
